# Expolinrg the equivalent damping equation $B_e$

In [ ]:
# %load ../../imports.py
"""
These is the standard setup for the notebooks.
"""

%matplotlib inline
%load_ext autoreload
%autoreload 2

from jupyterthemes import jtplot
jtplot.style(theme='onedork', context='notebook', ticks=True, grid=False)

import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)
import numpy as np
import os
import matplotlib.pyplot as plt
#plt.style.use('paper')

#import data
import copy
from rolldecay.bis_system import BisSystem
from rolldecay import database
from mdldb.tables import Run

from sklearn.pipeline import Pipeline
from rolldecayestimators.transformers import CutTransformer, LowpassFilterDerivatorTransformer, ScaleFactorTransformer, OffsetTransformer
from rolldecayestimators.direct_estimator_cubic import EstimatorQuadraticB, EstimatorCubic
from rolldecayestimators.ikeda_estimator import IkedaQuadraticEstimator
import rolldecayestimators.equations as equations
import rolldecayestimators.lambdas as lambdas
from rolldecayestimators.substitute_dynamic_symbols import lambdify
import rolldecayestimators.symbols as symbols
import sympy as sp

from sklearn.metrics import r2_score
import rolldecay.paper_writing as paper_writing



In [ ]:
from rolldecayestimators import measure as measure

## Is this a good linearization?

In [ ]:
equations.B_e_equation

In [ ]:
A_44 = 1.0
B_1=0.10
B_2=0.8
C_1=2.0

B_1A=B_1/A_44
B_2A=B_2/A_44
C_1A=C_1/A_44

estimator = EstimatorQuadraticB.load(B_1A=B_1A, B_2A=B_2A, C_1A=C_1A)

In [ ]:
N=100000
t=np.linspace(0,100.0,N)
phi0=np.deg2rad(10)
phi1d0=0.0

result = estimator.simulate(t=t, phi0=phi0, phi1d0=phi1d0)

In [ ]:
fig,ax=plt.subplots()
result.plot(y='phi', ax=ax)
ax.grid(True)

In [ ]:
amplitudes = measure.calculate_amplitudes_and_damping(X=result)
amplitudes['phi_a']=amplitudes['phi']/2
amplitudes['-phi_a']=-amplitudes['phi_a']
amplitudes.head()

In [ ]:
equations.omega0_equation

In [ ]:
omega0_lambda = lambdify(sp.solve(equations.omega0_equation,symbols.omega0)[0])

In [ ]:
omega0=omega0_lambda(A_44=A_44, C=C_1)
omega0

In [ ]:
2*np.pi/omega0

In [ ]:
equations.extinction_equation

In [ ]:
equations.xeta_equation

In [ ]:
zeta_lambda = lambdify(sp.solve(equations.xeta_equation,symbols.zeta)[0])

phi0=amplitudes.iloc[0]['phi_a']
t_=amplitudes.index-amplitudes.index[0]
amplitudes['zeta']=zeta_lambda(omega0=omega0, phi_0=phi0, phi_a=amplitudes['phi_a'], 
                               t=t_)

In [ ]:
amplitudes

In [ ]:
fig,ax=plt.subplots()
result.plot(y='phi', ax=ax)


amplitudes.plot( y='phi_a', ax=ax)
amplitudes.plot(y='-phi_a', ax=ax);


In [ ]:
zeta_lambda

In [ ]:
fig,ax=plt.subplots()
amplitudes.plot(x='phi_a', y=['B_n'], style='.-', ax=ax)

phi_a = amplitudes['phi_a']
y=B_1+(8/(3*np.pi)*omega0)*B_2*phi_a
ax.plot(phi_a,y,'.--', label='B_e')

ylim=ax.get_ylim()
ax.set_ylim([0,ylim[1]])
ax.legend()

In [ ]:
zero_crossings = measure.get_zerocrossings(result)

mask = zero_crossings['phi'] < 0
negative_peaks = zero_crossings.loc[mask].copy()
positive_peaks = zero_crossings.loc[~mask].copy()


In [ ]:
fig,ax=plt.subplots()
result.plot(y='phi', ax=ax)
negative_peaks.plot(y='phi', style='r.', ax=ax)
positive_peaks.plot(y='phi', style='g.', ax=ax)
ax.grid(True)


In [ ]:
positive_peaks

In [ ]:
df_decrements_positive = positive_peaks.shift()/positive_peaks
df_decrements_negative = negative_peaks.shift()/negative_peaks
df_decrements = pd.concat([df_decrements_positive,df_decrements_negative])
df_decrements.sort_index(inplace=True)

In [ ]:
zero_crossings['B_n'] = 2*(1 / (2 * np.pi) * np.log(df_decrements['phi']))

In [ ]:
zero_crossings['phi_a']=zero_crossings['phi'].abs()*(1+df_decrements['phi'])/2

In [ ]:
zero_crossings

In [ ]:
fig,ax=plt.subplots()
zero_crossings.plot(x='phi_a', y=['B_n'], style='.-', ax=ax)

phi_a = zero_crossings['phi_a']
y=B_1+(8/(3*np.pi)*omega0)*B_2*phi_a
ax.plot(phi_a,y,'.--', label='B_e')

ylim=ax.get_ylim()
ax.set_ylim([0,ylim[1]])
ax.legend()